In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm

In [ ]:
covid_train = pd.read_csv('data/covid/covid.train.csv').values
covid_test = pd.read_csv('data/covid/covid.test.csv').values
covid_train.shape

## Data Preprocessing

In [ ]:
def train_valid_split(train_data,ratio,seed=42):
    np.random.seed(seed)
    rand_index = np.random.permutation(len(train_data))
    vaild_index = rand_index[:int(len(train_data)*ratio)]
    train_index = rand_index[int(len(train_data)*ratio):]
    return train_data[train_index],train_data[vaild_index]
    
def select_feat(train_data, valid_data, test_data,):
    y_train,y_vaild = train_data[:,-1],valid_data[:,-1]
    x_train,x_vaild,x_test = train_data[:,:-1],valid_data[:,:-1],test_data
    return x_train,x_vaild,x_test,y_train,y_vaild
train_data, valid_data = train_valid_split(covid_train,.2)
x_train,x_vaild,x_test,y_train,y_vaild = select_feat(train_data, valid_data,covid_test)
x_train.shape,x_vaild.shape,x_test.shape

In [ ]:
class covid_dataset(Dataset):
    
    def __init__(self,x,y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __getitem__(self,idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx],self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [ ]:
train_loader = DataLoader(covid_dataset(x_train,y_train),batch_size=batch)
vaild_loader = DataLoader(covid_dataset(x_vaild,y_vaild),batch_size=batch)
test_loader = DataLoader(covid_dataset(x_test),batch_size=batch)
for x,y in train_loader:
    print(f'x:{x.shape}\ny:{y.shape}')
    break

## Network Model

In [ ]:
class net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(117,50),
            torch.nn.ReLU(),
            torch.nn.Linear(50,8),
            torch.nn.ReLU(),
            torch.nn.Linear(8,1),
        )
    
    def forward(self,x):
        x = self.layers(x)
        x = x.squeeze()
        return x
model = net().to('cuda')

#### *下面有很长一部分的训练代码，此处只提出第一步就出问题的部分

#### *训练几轮之后loss会一直变大直到inf

In [ ]:
loss_f = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

for x,y in train_loader:
    x,y = x.to('cuda'),y.to('cuda')
    pred = model(x)
    loss = loss_f(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'LOSS:{loss.item()}')
    print(f'PRDE:{pred},\nY:{y}')#标注了预测结果和目标值，到最后pred会摆烂输出一样的数